In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_landsat()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_h,wp_le,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,2.605590,13.643902,0.377924,0.413754,0.594708,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,3.459300,13.498761,0.381184,0.420529,0.597960,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,2.117474,14.987823,0.384444,0.427304,0.601212,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,4.636145,11.355851,0.387704,0.434079,0.604465,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,3.448946,10.943102,0.390963,0.440854,0.607717,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 24 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 non-null float64
wp_LST.night    1028 non-null float64
LW_IN.wp        1028 non-null float64
LW_IN.si        1028 non-null float64
LW_IN           1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_h"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.917747685181
CV Scores:  0.920848711378, 0.920702456157, 0.932956959984, 0.89625498674, 0.932566715718, 0.925741792524, 0.914755800039, 0.92670497009, 0.902699662082, 0.904244797103
OOB score: 0.922893713909
Feature Importances:
('sw_in', 0.20423262367406889)
('wp_ndvi', 0.12376697147465275)
('wp_lswi2', 0.11278425740870837)
('PET', 0.095751044931105783)
('wp_evi', 0.09189967513188059)
('VPD', 0.084786960755221685)
('wp_LST.day', 0.067344134382007995)
('air_temp', 0.063827611293925599)
('wp_LST.night', 0.056209742504614926)
('LW_IN.wp', 0.044135748013134857)
('wind_speed', 0.041235431114816062)
('precip', 0.014025799315862442)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.934172430623
CV Scores:  0.950714981904, 0.934436165324, 0.945459041685, 0.920713067944, 0.938310206301, 0.931728522216, 0.926986204427, 0.941854333949, 0.926719397489, 0.924802384994
Feature Importances:
('sw_in', 0.15347326203384176)
('VPD', 0.12371180368714707)
('wind_speed', 0.11173034081421074)
('PET', 0.10683989320416529)
('wp_ndvi', 0.097730083499419071)
('wp_LST.day', 0.089434298798888673)
('wp_lswi2', 0.07349324771164327)
('LW_IN.wp', 0.066502665471594599)
('wp_evi', 0.059251989619016253)
('wp_LST.night', 0.058062323478460652)
('air_temp', 0.038875514718852672)
('precip', 0.02089457696276004)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.939067588584
CV Scores:  0.955776690439, 0.921232389488, 0.948918515944, 0.928069094581, 0.951394893727, 0.92071374114, 0.923009272324, 0.958002593497, 0.939670139225, 0.943888555475


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 15.01643
Step #501, epoch #50, avg. loss: 0.77769
Step #1001, epoch #100, avg. loss: 0.32889
Step #1501, epoch #150, avg. loss: 0.24697
Step #2001, epoch #200, avg. loss: 0.21583
Step #2501, epoch #250, avg. loss: 0.19095
Step #3001, epoch #300, avg. loss: 0.17389
Step #3501, epoch #350, avg. loss: 0.15550
Step #4001, epoch #400, avg. loss: 0.14660
Step #4501, epoch #450, avg. loss: 0.13108
Step #1, avg. loss: 13.12240
Step #501, epoch #50, avg. loss: 0.82951
Step #1001, epoch #100, avg. loss: 0.34281
Step #1501, epoch #150, avg. loss: 0.28391
Step #2001, epoch #200, avg. loss: 0.23171
Step #2501, epoch #250, avg. loss: 0.20457
Step #3001, epoch #300, avg. loss: 0.17930
Step #3501, epoch #350, avg. loss: 0.16874
Step #4001, epoch #400, avg. loss: 0.15190
Step #4501, epoch #450, avg. loss: 0.14553
Step #1, avg. loss: 12.65608
Step #501, epoch #50, avg. loss: 0.82272
Step #1001, epoch #100, avg. loss: 0.35744
Step #1501, epoc

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x1139f4488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)